In [1]:
#second protocol 

In [13]:
import qiskit
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
import numpy as np
import csv

def prepare_initial_state(qc, qubit):
    qc.h(qubit)  # Hadamard gate to prepare |+⟩ state

def prepare_sift_state(qc, qubit, cbit):
    qc.measure(qubit, cbit)  # Measure the qubit in Z basis
    qc.reset(qubit)  # Reset the qubit
    # Resend a new qubit based on the measurement result
    qc.x(qubit).c_if(cbit, 1)

def prepare_eve_state(qc, qubit):
    choice = np.random.randint(4)  # Randomly choose among four different states
    if choice == 0:
        qc.initialize([1, 0], qubit)  # Prepare |0⟩ state
    elif choice == 1:
        qc.initialize([0, 1], qubit)  # Prepare |1⟩ state
    elif choice == 2:
        qc.h(qubit)  # Prepare |+⟩ state
    elif choice == 3:
        qc.x(qubit)
        qc.h(qubit)  # Prepare |-⟩ state

def create_msqkd_circuit(num_qubits, eavesdropper_present=False):
    qc = QuantumCircuit(num_qubits * 2, num_qubits * 2)
    operations = []

    # Step 1: TP prepares the initial states
    for i in range(num_qubits):
        prepare_initial_state(qc, i)  # Alice's qubit
        prepare_initial_state(qc, num_qubits + i)  # Bob's qubit

    # Eavesdropper actions
    if eavesdropper_present:
        for i in range(num_qubits):
            prepare_eve_state(qc, i)  # Eve prepares a state on Alice's qubit
            prepare_eve_state(qc, num_qubits + i)  # Eve prepares a state on Bob's qubit

    for i in range(num_qubits):
        # Step 2: Alice and Bob decide the operation (CTRL or SIFT)
        alice_operation = np.random.choice(['SIFT', 'CTRL'])
        bob_operation = np.random.choice(['SIFT', 'CTRL'])
        operations.append((alice_operation, bob_operation))

        if alice_operation == 'SIFT':
            prepare_sift_state(qc, i, i)
        # CTRL operation does nothing to the qubit

        if bob_operation == 'SIFT':
            prepare_sift_state(qc, num_qubits + i, num_qubits + i)
        # CTRL operation does nothing to the qubit

    # Step 4: Measure the qubits in the Z basis
    for i in range(num_qubits):
        qc.measure([i, num_qubits + i], [i, num_qubits + i])

    return qc, operations

def run_simulation(num_qubits, noise_level_single, noise_level_two, eavesdropper_present=False):
    noise_model = NoiseModel()
    error_single_qubit = depolarizing_error(noise_level_single, 1)
    error_two_qubit = depolarizing_error(noise_level_two, 2)
    noise_model.add_all_qubit_quantum_error(error_single_qubit, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(error_two_qubit, ['cx'])

    qc, operations = create_msqkd_circuit(num_qubits, eavesdropper_present)
    simulator = Aer.get_backend('qasm_simulator')
    compiled_circuit = qiskit.transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1024, noise_model=noise_model)
    result = job.result()
    counts = result.get_counts()

    alice_ops = [op[0] for op in operations]
    bob_ops = [op[1] for op in operations]

    raw_key = []
    sift_results = []
    ctrl_results = []
    sift_correlations = 0
    total_sift_pairs = 0

    for key, count in counts.items():
        bits = [int(bit) for bit in key]
        for i in range(num_qubits):
            if alice_ops[i] == 'SIFT' and bob_ops[i] == 'SIFT':
                total_sift_pairs += 1
                expected_bell_state = f"{bits[i]}{bits[num_qubits + i]}"
                actual_bell_state = f"{key[i]}{key[num_qubits + i]}"
                sift_results.append((bits[i], bits[num_qubits + i], key))
                if expected_bell_state == actual_bell_state:
                    raw_key.append(bits[i])
                
            if alice_ops[i] == 'CTRL' and bob_ops[i] == 'CTRL':
                ctrl_results.append((bits[i], bits[num_qubits + i], key))

    if total_sift_pairs > 0:
        check_indices = np.random.choice(total_sift_pairs, total_sift_pairs // 2, replace=False)
        for idx in check_indices:
            if idx < len(bits) and (num_qubits + idx) < len(bits):
                expected_bell_state = f"{bits[idx]}{bits[num_qubits + idx]}"
                actual_bell_state = f"{key[idx]}{key[num_qubits + idx]}"
                sift_results.append((bits[idx], bits[num_qubits + idx], key))
                if expected_bell_state == actual_bell_state:
                    sift_correlations += 1

    sift_correlation_rate = sift_correlations / (total_sift_pairs / 2) if total_sift_pairs > 0 else 0
    eavesdropper_detected = (sift_correlation_rate < 0.75)

    final_key = raw_key
    key_generation_rate = len(raw_key) / num_qubits
    final_key_length = len(final_key)

    return {
        "Num_Qubits": num_qubits,
        "Noise_Level_Single": noise_level_single,
        "Noise_Level_Two": noise_level_two,
        "Eavesdropper_Present": eavesdropper_present,
        "Key_Generation_Rate": key_generation_rate,
        "Final_Key_Length": final_key_length,
        "Eavesdropper_Detected": eavesdropper_detected,
    }

# Run the simulation for different parameters and save the results to CSV
num_runs = 1
num_qubits_list = [4,6,8,10,12,14]
noise_levels_single = [0,0.001, 0.005, 0.01,0.05,0.1]  # Noise levels for single-qubit gates
noise_levels_two = [0,0.01, 0.05, 0.1,0.5,1.0]  # Noise levels for two-qubit gates


with open('msqkd_sim_results2.csv', 'w', newline='') as csvfile:
    fieldnames = ["Num_Qubits", "Noise_Level_Single", "Noise_Level_Two", "Eavesdropper_Present", "Key_Generation_Rate", "Final_Key_Length", "Eavesdropper_Detected"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    total_simulations = num_runs * len(num_qubits_list) * len(noise_levels_single) * len(noise_levels_two) * 2
    current_simulation = 0

    for num_qubits in num_qubits_list:
        for noise_level_single in noise_levels_single:
            for noise_level_two in noise_levels_two:
                for eavesdropper_present in [True, False]:
                    for _ in range(num_runs):
                        result = run_simulation(num_qubits, noise_level_single, noise_level_two, eavesdropper_present)
                        writer.writerow(result)
                        current_simulation += 1
                        print(f"Progress: {current_simulation}/{total_simulations} simulations completed")

Progress: 1/432 simulations completed
Progress: 2/432 simulations completed
Progress: 3/432 simulations completed
Progress: 4/432 simulations completed
Progress: 5/432 simulations completed
Progress: 6/432 simulations completed
Progress: 7/432 simulations completed
Progress: 8/432 simulations completed
Progress: 9/432 simulations completed
Progress: 10/432 simulations completed
Progress: 11/432 simulations completed
Progress: 12/432 simulations completed
Progress: 13/432 simulations completed
Progress: 14/432 simulations completed
Progress: 15/432 simulations completed
Progress: 16/432 simulations completed
Progress: 17/432 simulations completed
Progress: 18/432 simulations completed
Progress: 19/432 simulations completed
Progress: 20/432 simulations completed
Progress: 21/432 simulations completed
Progress: 22/432 simulations completed
Progress: 23/432 simulations completed
Progress: 24/432 simulations completed
Progress: 25/432 simulations completed
Progress: 26/432 simulations compl

Progress: 209/432 simulations completed
Progress: 210/432 simulations completed
Progress: 211/432 simulations completed
Progress: 212/432 simulations completed
Progress: 213/432 simulations completed
Progress: 214/432 simulations completed
Progress: 215/432 simulations completed
Progress: 216/432 simulations completed
Progress: 217/432 simulations completed
Progress: 218/432 simulations completed
Progress: 219/432 simulations completed
Progress: 220/432 simulations completed
Progress: 221/432 simulations completed
Progress: 222/432 simulations completed


KeyboardInterrupt: 